Pipeline kéo dữ liệu từ FACEBOOK & INSTAGRAM (Meta Graph API)

In [13]:
import os, csv, time, requests

USER_TOKEN   = "EAASa6oZC3IFwBPIuQoKpcWIjfavWbV4sT9uBf7afijEKvJVaEeyfZBbIpPxZBpCTPkVbfuZBOicipcEJxYznd5jtpPjaAwbhiOINbFO7FfsUNp17LmEEPGKyHPvNNZCfg6gxitLqBt3xrPRK3RjFyge5T5PDXM3FZA3gWihyY8ZCwEVEdEc49rKZAtWRYjThRxn0txZCxrutzPIIKZBPwywCivznsTnE2J5IZBpTdhx"
PAGE_ID      = "178312595372256"   # ví dụ: "1000123456789"
OUT_POSTS    = r"c:\Users\Admin\Downloads\fb_posts.csv"
OUT_COUNTS   = r"c:\Users\Admin\Downloads\fb_counts.csv"

def fb_get(url, params):
    r = requests.get(url, params=params, timeout=30)
    if r.status_code >= 400:
        try:
            print("Graph error:", r.json())
        except Exception:
            print("Raw error:", r.text[:500])
        r.raise_for_status()
    return r.json()

def get_page_access_token(user_token, page_id):
    # Lấy danh sách page mà user quản trị + page token tương ứng
    data = fb_get("https://graph.facebook.com/v18.0/me/accounts",
                  {"access_token": user_token, "limit": 200})
    for item in data.get("data", []):
        if str(item.get("id")) == str(page_id):
            return item.get("access_token")
    raise RuntimeError("Không tìm thấy PAGE_ID trong /me/accounts. Kiểm tra PAGE_ID hoặc quyền của user.")

def fetch_all(url, params):
    while True:
        data = fb_get(url, params)
        for it in data.get("data", []):
            yield it
        paging = data.get("paging", {})
        next_url = paging.get("next")
        if not next_url:
            break
        # next đã chứa token/tham số
        url, params = next_url, {}

def fetch_page_posts(page_id, page_token, limit=100):
    url = f"https://graph.facebook.com/v18.0/{page_id}/posts"
    params = {
        "fields": "id,message,created_time,permalink_url",
        "limit": limit,
        "access_token": page_token
    }
    yield from fetch_all(url, params)

def fetch_counts(post_id, page_token):
    url = f"https://graph.facebook.com/v18.0/{post_id}"
    params = {
        "fields": "reactions.summary(true).limit(0),comments.summary(true).limit(0)",
        "access_token": page_token
    }
    data = fb_get(url, params)
    likes = (data.get("reactions") or {}).get("summary", {}).get("total_count", 0)
    comments = (data.get("comments") or {}).get("summary", {}).get("total_count", 0)
    return likes, comments

def write_csv(path, rows, headers):
    exists = os.path.exists(path)
    with open(path, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=headers)
        if not exists:
            w.writeheader()
        for r in rows:
            w.writerow(r)

# --- MAIN ---
def main():
    if not USER_TOKEN:
        raise RuntimeError("Thiếu META_USER_TOKEN. Hãy export META_USER_TOKEN trước khi chạy.")
    page_token = get_page_access_token(USER_TOKEN, PAGE_ID)

    # 1) Kéo post
    posts = list(fetch_page_posts(PAGE_ID, page_token, limit=50))
    write_csv(OUT_POSTS, posts, ["id","message","created_time","permalink_url"])
    print(f"Đã ghi {len(posts)} post vào {OUT_POSTS}")

    # 2) Đếm like/comment cho từng post
    rows = []
    for p in posts:
        pid = p.get("id")
        try:
            likes, comments = fetch_counts(pid, page_token)
            rows.append({"post_id": pid, "likes": likes, "comments": comments})
            time.sleep(0.15)
        except Exception as e:
            print("Lỗi đếm post", pid, e)
    write_csv(OUT_COUNTS, rows, ["post_id","likes","comments"])
    print(f"Đã ghi {len(rows)} dòng count vào {OUT_COUNTS}")

if __name__ == "__main__":
    main()


Đã ghi 24 post vào c:\Users\Admin\Downloads\fb_posts.csv
Đã ghi 24 dòng count vào c:\Users\Admin\Downloads\fb_counts.csv


Lấy like/comment từ URL post công khai (FB/IG)

In [1]:
import json, re, csv, requests
from pathlib import Path
from bs4 import BeautifulSoup
from typing import Optional, Tuple, Dict, List
from urllib.parse import urlparse

# ========== CONFIG ==========
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    ),
    # Nếu cần, mở comment và dán cookie đăng nhập của bạn:
    # "Cookie": "ds_user_id=...; sessionid=...; csrftoken=...; datr=...; ...",
    "Accept-Language": "vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7",
}

LIKE_PATTERNS = [
    r'(?P<num>[\d,\. ]+)\s*likes?',
    r'(?P<num>[\d,\. ]+)\s*reactions?',
    r'(?P<num>[\d,\. ]+)\s*lượt\s*thích',
    r'(?P<num>[\d,\. ]+)\s*thích',
]
COMMENT_PATTERNS = [
    r'(?P<num>[\d,\. ]+)\s*comments?',
    r'(?P<num>[\d,\. ]+)\s*bình\s*luận',
]

LIKE_KEYWORDS = ["lượt thích", "thích", "likes", "reactions"]
COMMENT_KEYWORDS = ["bình luận", "comments"]

# ========== HELPERS ==========

def _to_int(s: Optional[str]) -> Optional[int]:
    """Parse '1.2k', '3.4m', '12,345' -> int"""
    if s is None:
        return None
    s = s.strip().lower()

    # 1) 1.2k / 3.4m
    m = re.fullmatch(r'([\d]+(?:\.\d+)?)([km])', s)
    if m:
        val = float(m.group(1))
        unit = m.group(2)
        return int(val * (1_000 if unit == 'k' else 1_000_000))

    # 2) số nguyên có phân cách
    s2 = re.sub(r'[\s,]', '', s)            # bỏ space & phẩy
    s2 = re.sub(r'\.(?=\d{3}(\D|$))', '', s2)  # bỏ chấm nếu là phân cách nghìn
    return int(s2) if re.fullmatch(r'\d+', s2) else None

def parse_jsonld_interactions(soup: BeautifulSoup) -> Tuple[Optional[int], Optional[int]]:
    likes = comments = None
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string or "")
        except Exception:
            continue
        items = data if isinstance(data, list) else [data]
        for it in items:
            stats = it.get("interactionStatistic") or []
            if isinstance(stats, dict):
                stats = [stats]
            for st in stats:
                itype = (st.get("interactionType") or "")
                count = st.get("userInteractionCount")
                if isinstance(itype, str):
                    itype_l = itype.lower()
                    if likes is None and "like" in itype_l:
                        likes = _to_int(str(count)) if count is not None else likes
                    if comments is None and "comment" in itype_l:
                        comments = _to_int(str(count)) if count is not None else comments
    return likes, comments

def regex_text_for_counts(text: str) -> Tuple[Optional[int], Optional[int]]:
    likes = comments = None
    for pat in LIKE_PATTERNS:
        m = re.search(pat, text, flags=re.I)
        if m:
            likes = _to_int((m.group("num") or "").strip())
            break
    for pat in COMMENT_PATTERNS:
        m = re.search(pat, text, flags=re.I)
        if m:
            comments = _to_int((m.group("num") or "").strip())
            break
    return likes, comments

def _near_keyword_number_spans(soup: BeautifulSoup,
                               keywords: List[str]) -> Optional[int]:
    """
    Tìm <span> số nằm trong vùng văn bản có chứa 1 từ khóa bất kỳ.
    Ví dụ DOM IG/FB: <span>31</span> ... "lượt thích"
    """
    # 1) tìm các span chỉ chứa số hoặc số + k/m
    numeric_spans = soup.find_all(
        "span", string=re.compile(r"^\s*\d[\d\.,\s]*([km])?\s*$", re.I)
    )
    for sp in numeric_spans:
        # gom text cha (nhiều cấp), dừng ở container vừa phải
        parent = sp
        hops = 0
        found = False
        while parent and hops < 4:
            parent = parent.parent
            hops += 1
            if not parent:
                break
            pt = parent.get_text(" ", strip=True).lower()
            # nếu trong block có keyword thì lấy số
            if any(kw in pt for kw in keywords):
                val = _to_int(sp.get_text(strip=True))
                if val is not None:
                    return val
                # nếu span là "31" nhưng đơn vị ở sibling, thử bắt cụm gần kề
                sib_text = " ".join(s.get_text(" ", strip=True).lower()
                                    for s in parent.find_all("span"))
                if any(kw in sib_text for kw in keywords):
                    val = _to_int(sp.get_text(strip=True))
                    if val is not None:
                        return val
    return None

def parse_dom_counts(soup: BeautifulSoup) -> Tuple[Optional[int], Optional[int]]:
    """Chiến lược DOM: bắt số gần keywords."""
    likes = _near_keyword_number_spans(soup, LIKE_KEYWORDS)
    comments = _near_keyword_number_spans(soup, COMMENT_KEYWORDS)
    return likes, comments

# ========== CORE ==========

def extract_like_comment_counts(url: str) -> Dict[str, Optional[int]]:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    html = r.text
    soup = BeautifulSoup(html, "html.parser")

    likes = comments = None

    # 1) JSON-LD
    l1, c1 = parse_jsonld_interactions(soup)
    likes = likes if likes is not None else l1
    comments = comments if comments is not None else c1

    # 2) og:description
    if likes is None or comments is None:
        og = soup.find("meta", attrs={"property": "og:description"})
        if og and og.get("content"):
            l2, c2 = regex_text_for_counts(og["content"])
            likes = likes if likes is not None else l2
            comments = comments if comments is not None else c2

    # 3) Regex toàn trang (fallback)
    if likes is None or comments is None:
        l3, c3 = regex_text_for_counts(soup.get_text(" ", strip=True))
        likes = likes if likes is not None else l3
        comments = comments if comments is not None else c3

    # 4) DOM số + từ khóa (mới)
    if likes is None or comments is None:
        l4, c4 = parse_dom_counts(soup)
        likes = likes if likes is not None else l4
        comments = comments if comments is not None else c4

    # ghi chú lý do nếu cả 2 vẫn None (hay gặp với IG/FB ko có cookie)
    if likes is None and comments is None:
        host = urlparse(url).hostname or ""
        note = None
        if "instagram.com" in host or "facebook.com" in host:
            note = "Site không để lộ số trong HTML tĩnh. Cần cookie đăng nhập/Graph API."
        else:
            note = "Không tìm thấy số like/comment trong HTML hoặc metadata."
        return {"url": url, "likes": None, "comments": None, "note": note}

    return {"url": url, "likes": likes, "comments": comments}

def save_to_csv(results: List[Dict], filename: str = "fb_ig_posts.csv") -> None:
    with open(filename, "w", newline="", encoding="utf-8") as f:
        fieldnames = ["url", "likes", "comments", "note"]
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for row in results:
            if "note" not in row:
                row["note"] = ""
            w.writerow(row)
    print(f"✅ Đã lưu {len(results)} dòng vào {filename}")

# ========== CLI DEMO ==========

if __name__ == "__main__":
    urls = [
        # Ví dụ IG (cần cookie đăng nhập để có HTML đầy đủ)
        "https://www.instagram.com/reel/DNRt3h_Bjk3/",
        # Thêm các URL FB/IG công khai khác...
    ]
    results = [extract_like_comment_counts(u) for u in urls]

    for r in results:
        print(r)

    # Xuất CSV
    save_to_csv(results, r"C:\Users\Admin\Downloads\posts_stats.csv")


{'url': 'https://www.instagram.com/reel/DNRt3h_Bjk3/', 'likes': None, 'comments': None, 'note': 'Site không để lộ số trong HTML tĩnh. Cần cookie đăng nhập/Graph API.'}
✅ Đã lưu 1 dòng vào C:\Users\Admin\Downloads\posts_stats.csv
